<a href="https://colab.research.google.com/github/bpanny/nlp-hw4/blob/main/bmp83_hw4_bert_ner.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Fine-tune BERT-based models from Hugging Face on CoNLL-2002 Spanish NER data

In this notebook, you will fine-tune and evaluate multiple BERT-based models on CoNLL-2002 Spanish NER data.

Code for loading and preprocessing the data is provided. You will provide code for training and evaluation using Hugging Face Trainer or PyTorch.

Please copy this notebook and name it `{pitt email id}_hw4_bert_ner.ipynb`.

Run all the cells starting from the top, filling in any sections that need to be filled in. Spots you need to fill in are specified.

**Note**: Please run on GPU by going to Runtime > Change Runtime Type > T4 GPU

This notebook is based on:
* https://github.com/laxmimerit/NLP-Tutorials-with-HuggingFace/blob/main/NLP_with_HuggingFace_Tutorial_2_NER_Training.ipynb  
* https://skimai.com/how-to-fine-tune-bert-for-named-entity-recognition-ner/

# Set up environment, preprocess data

In [2]:
# Download and install needed Hugging Face packages

!pip install -U transformers
!pip install -U accelerate
!pip install -U datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.9/7.9 MB 13.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 311.2/311.2 kB 36.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 40.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 44.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 43.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 261.4/261.4 kB 3.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 493.7/493.7 kB 8.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 19.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 22.5 MB/s eta 0:00:00


In [3]:
# Load dataset, which contains splits for training, validation (dev), and test

import pandas as pd
from datasets import load_dataset

data = load_dataset('conll2002', 'es')
data

Extracting data files:   0%|          | 0/3 [00:00<?, ?it/s]

Generating train split:   0%|          | 0/8324 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/1916 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/1518 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['id', 'tokens', 'pos_tags', 'ner_tags'],
        num_rows: 8324
    })
    validation: Dataset({
        features: ['id', 'tokens', 'pos_tags', 'ner_tags'],
        num_rows: 1916
    })
    test: Dataset({
        features: ['id', 'tokens', 'pos_tags', 'ner_tags'],
        num_rows: 1518
    })
})

In [4]:
# Examine the tagset. Note the BIO framework with 4 possible types

tags = data['train'].features['ner_tags'].feature

index2tag = {idx:tag for idx, tag in enumerate(tags.names)}
tag2index = {tag:idx for idx, tag in enumerate(tags.names)}
index2tag

{0: 'O',
 1: 'B-PER',
 2: 'I-PER',
 3: 'B-ORG',
 4: 'I-ORG',
 5: 'B-LOC',
 6: 'I-LOC',
 7: 'B-MISC',
 8: 'I-MISC'}

In [5]:
# Put human-readable NER tags in data

def create_tag_names(batch):
  tag_name = {'ner_tags_str': [tags.int2str(idx) for idx in batch['ner_tags']]}
  return tag_name

data = data.map(create_tag_names)

Map:   0%|          | 0/8324 [00:00<?, ? examples/s]

Map:   0%|          | 0/1916 [00:00<?, ? examples/s]

Map:   0%|          | 0/1518 [00:00<?, ? examples/s]

In [6]:
# Take a look at the data
pd.DataFrame(data['train'])[['tokens', 'ner_tags', 'ner_tags_str']].head(3)

,tokens,ner_tags,ner_tags_str
0,"[Melbourne, (, Australia, ), ,, 25, may, (, EF...","[5, 0, 5, 0, 0, 0, 0, 0, 3, 0, 0]","[B-LOC, O, B-LOC, O, O, O, O, O, B-ORG, O, O]"
1,[-],[0],[O]
2,"[El, Abogado, General, del, Estado, ,, Daryl, ...","[0, 1, 2, 2, 2, 0, 1, 2, 0, 0, 0, 0, 0, 0, 0, ...","[O, B-PER, I-PER, I-PER, I-PER, O, B-PER, I-PE..."


## Metrics
Load NER-specific evaluation metrics

In [7]:
!pip install seqeval
!pip install evaluate

import evaluate
import numpy as np

metric = evaluate.load('seqeval')
ner_feature = data['train'].features['ner_tags']
label_names = ner_feature.feature.names
labels = data['train'][0]['ner_tags']
labels = [label_names[i] for i in labels]

def compute_metrics(eval_preds):
  logits, labels = eval_preds

  predictions = np.argmax(logits, axis=-1)

  true_labels = [[label_names[l] for l in label if l!=-100] for label in labels]

  true_predictions = [[label_names[p] for p,l in zip(prediction, label) if l!=-100]
                      for prediction, label in zip(predictions, labels)]

  all_metrics = metric.compute(predictions=true_predictions, references=true_labels)

  return {"precision": all_metrics['overall_precision'],
          "recall": all_metrics['overall_recall'],
          "f1": all_metrics['overall_f1'],
          "accuracy": all_metrics['overall_accuracy']}

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 2.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for seqeval: filename=seqeval-1.2.2-py3-none-any.whl size=16161 sha256=5bbc36e07e88b28f949824a5c30ec622a4fff023c1633dce8be9b12dffcdda51
  Stored in directory: /root/.cache/pip/wheels/1a/67/4a/ad4082dd7dfc30f2abfe4d80a2ed5926a506eb8a972b4767fa
Successfully built seqeval
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 1.6 MB/s eta 0:00:00


# Fine-tune models
This section is where you choose models and fill in parts of the code to do fine-tuning.

You need to fine-tune at least 2 pretrained models from the Hugging Face platform on the preprocessed CoNLL-2002 Spanish data:
* One BERT-based model pretrained with a regular masked language modeling (MLM) objective on a Spanish corpus. Examples: `PlanTL-GOB-ES/roberta-base-bne`, `chriskhanhtran/spanberta`
* One model pretrained to perform NER on another language, such as English. Models pretrained on the CoNLL-2003 dataset often work. Examples: `elastic/distilbert-base-cased-finetuned-conll03-english`, `dbmdz/bert-bert-cased-finetuned-conll03-english`

You'll want to make sure whatever pretrained model is cased, which contains valuable information for NER.

In [ ]:
# FILL IN which model you are fine-tuning and assign the name of it to the `pretrained_model` variable

pretrained_model = "chriskhanhtran/spanberta"
# pretrained_model = "elastic/distilbert-base-cased-finetuned-conll03-english"


In [ ]:
# Tokenize the data with the pretrained model's tokenizer
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained(pretrained_model, use_fast=True, add_prefix_space=True)

def align_labels_with_tokens(labels, word_ids):
  new_labels = []
  current_word=None
  for word_id in word_ids:
    if word_id != current_word:
      current_word = word_id
      label = -100 if word_id is None else labels[word_id]
      new_labels.append(label)

    elif word_id is None:
      new_labels.append(-100)

    else:
      label = labels[word_id]

      if label%2==1:
        label = label + 1
      new_labels.append(label)

  return new_labels

def tokenize_and_align_labels(examples):
  tokenized_inputs = tokenizer(examples['tokens'], truncation=True, is_split_into_words=True)

  all_labels = examples['ner_tags']

  new_labels = []
  for i, labels in enumerate(all_labels):
    word_ids = tokenized_inputs.word_ids(i)
    new_labels.append(align_labels_with_tokens(labels, word_ids))

  tokenized_inputs['labels'] = new_labels

  return tokenized_inputs
tokenized_datasets = data.map(tokenize_and_align_labels, batched=True, remove_columns=data['train'].column_names)
tokenized_datasets

Map:   0%|          | 0/8324 [00:00<?, ? examples/s]

Map:   0%|          | 0/1916 [00:00<?, ? examples/s]

Map:   0%|          | 0/1518 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 8324
    })
    validation: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 1916
    })
    test: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 1518
    })
})

In [ ]:
# Build a data collator to handle batching

from transformers import DataCollatorForTokenClassification

data_collator = DataCollatorForTokenClassification(tokenizer=tokenizer)

# Train (fine-tune) the model

In [ ]:
id2label = {i:label for i, label in enumerate(label_names)}
label2id = {label:i for i, label in enumerate(label_names)}
print(id2label)

{0: 'O', 1: 'B-PER', 2: 'I-PER', 3: 'B-ORG', 4: 'I-ORG', 5: 'B-LOC', 6: 'I-LOC', 7: 'B-MISC', 8: 'I-MISC'}


In [ ]:
from transformers import AutoModelForTokenClassification

model = AutoModelForTokenClassification.from_pretrained(pretrained_model,
                                                    id2label=id2label,
                                                    label2id=label2id)

Some weights of RobertaForTokenClassification were not initialized from the model checkpoint at chriskhanhtran/spanberta and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


## FILL IN code to train
Provide code to train (fine-tune) the pretrained model.
 You can use Hugging Face Trainer class or use any other package you want, such as PyTorch.

 See the [Hugging Face Trainer user guide](https://huggingface.co/learn/nlp-course/chapter3/3?fw=pt) or use any other online examples/resources you find online.

In [ ]:
from transformers import TrainingArguments

training_args = TrainingArguments("test-trainer")

In [ ]:
tokenized_datasets

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 8324
    })
    validation: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 1916
    })
    test: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 1518
    })
})

In [ ]:
from transformers import Trainer

trainer = Trainer(
    model,
    training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics = compute_metrics
)

In [ ]:
trainer.train()

Step,Training Loss
500,0.035800
1000,0.035200
1500,0.029700
2000,0.026100
2500,0.022400
3000,0.019900


TrainOutput(global_step=3123, training_loss=0.028216188090602595, metrics={'train_runtime': 494.001, 'train_samples_per_second': 50.551, 'train_steps_per_second': 6.322, 'total_flos': 899666963201448.0, 'train_loss': 0.028216188090602595, 'epoch': 3.0})

# Evaluate the fine-tuned model

## FILL IN code to evaluate performance of the model on the test set
Provide code to evaluate the pretrained model on the `test` portion of the dataset (`tokenized_datasets['test']`)

You'll need the F1 score for your report.
This is calculated automatically if you passed the `compute_metrics` function to the `Trainer` class.

In [ ]:
# Testing code here
predictions = trainer.predict(tokenized_datasets["test"])

In [ ]:
predictions[-1]

{'test_loss': 0.12180974334478378,
 'test_precision': 0.723716990533134,
 'test_recall': 0.8162405169991571,
 'test_f1': 0.7671992605308332,
 'test_accuracy': 0.9765721331689272,
 'test_runtime': 10.6394,
 'test_samples_per_second': 142.677,
 'test_steps_per_second': 17.858}

The F1 score for "chriskhanhtran/spanberta" is 0.7671992605308332

Hooray, you're done evaluating a model!

Feel free to restart the runtime and evaluate another one, or test that model on an example in the section below (which you'll need to do for at least one model).

# Test the model on an example
Code is provided here to test your fine-tuned classifier on an example sentence.

You will need to fill in the path to a checkpoint of your fine-tuned model if it has been saved somewhere. Or feel free to run your model some other way on the example sentence.

You will need the output of running at least one of your models on the example sentence for your report.

In [ ]:
# Test performance on an example

from transformers import pipeline

checkpoint = "/content/test-trainer/checkpoint-3000" # FILL IN path to one of the checkpoints of your fine-tuned model
token_classifier = pipeline(
    "token-classification", model=checkpoint, aggregation_strategy="simple"
)

test_sentence = "Mi nombre is Miguel Salgado. Trabajo en la Universidad de Pittsburgh y vivo en Pittsburgh."
token_classifier(test_sentence)

[{'entity_group': 'PER',
  'score': 0.99776167,
  'word': ' Miguel Salgado.',
  'start': 13,
  'end': 28},
 {'entity_group': 'ORG',
  'score': 0.5067352,
  'word': ' Universidad',
  'start': 43,
  'end': 54},
 {'entity_group': 'LOC',
  'score': 0.82354796,
  'word': ' de Pittsburgh',
  'start': 55,
  'end': 68},
 {'entity_group': 'LOC',
  'score': 0.99871534,
  'word': ' Pittsburgh.',
  'start': 79,
  'end': 90}]

The model correctly identifies Miguel as a person, correctly identifies University as an organization, but not University of Pittsburgh, It labels de Pittsburgh and Pittsburgh as locations. So it is right if I say it has labelled an organization of a location, but not right if I want it to label the organization of a location as a singular organizational entity.

In [10]:
# FILL IN which model you are fine-tuning and assign the name of it to the `pretrained_model` variable

pretrained_model = "chriskhanhtran/spanberta"
# pretrained_model = "elastic/distilbert-base-cased-finetuned-conll03-english"


In [11]:
# Tokenize the data with the pretrained model's tokenizer
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained(pretrained_model, use_fast=True, add_prefix_space=True)

def align_labels_with_tokens(labels, word_ids):
  new_labels = []
  current_word=None
  for word_id in word_ids:
    if word_id != current_word:
      current_word = word_id
      label = -100 if word_id is None else labels[word_id]
      new_labels.append(label)

    elif word_id is None:
      new_labels.append(-100)

    else:
      label = labels[word_id]

      if label%2==1:
        label = label + 1
      new_labels.append(label)

  return new_labels

def tokenize_and_align_labels(examples):
  tokenized_inputs = tokenizer(examples['tokens'], truncation=True, is_split_into_words=True)

  all_labels = examples['ner_tags']

  new_labels = []
  for i, labels in enumerate(all_labels):
    word_ids = tokenized_inputs.word_ids(i)
    new_labels.append(align_labels_with_tokens(labels, word_ids))

  tokenized_inputs['labels'] = new_labels

  return tokenized_inputs
tokenized_datasets = data.map(tokenize_and_align_labels, batched=True, remove_columns=data['train'].column_names)
tokenized_datasets

Map:   0%|          | 0/8324 [00:00<?, ? examples/s]

Map:   0%|          | 0/1916 [00:00<?, ? examples/s]

Map:   0%|          | 0/1518 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 8324
    })
    validation: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 1916
    })
    test: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 1518
    })
})

In [12]:
# Build a data collator to handle batching

from transformers import DataCollatorForTokenClassification

data_collator = DataCollatorForTokenClassification(tokenizer=tokenizer)

# Train (fine-tune) the model

In [13]:
id2label = {i:label for i, label in enumerate(label_names)}
label2id = {label:i for i, label in enumerate(label_names)}
print(id2label)

{0: 'O', 1: 'B-PER', 2: 'I-PER', 3: 'B-ORG', 4: 'I-ORG', 5: 'B-LOC', 6: 'I-LOC', 7: 'B-MISC', 8: 'I-MISC'}


In [14]:
from transformers import AutoModelForTokenClassification

model = AutoModelForTokenClassification.from_pretrained(pretrained_model,
                                                    id2label=id2label,
                                                    label2id=label2id)

Some weights of RobertaForTokenClassification were not initialized from the model checkpoint at chriskhanhtran/spanberta and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


## FILL IN code to train
Provide code to train (fine-tune) the pretrained model.
 You can use Hugging Face Trainer class or use any other package you want, such as PyTorch.

 See the [Hugging Face Trainer user guide](https://huggingface.co/learn/nlp-course/chapter3/3?fw=pt) or use any other online examples/resources you find online.

In [17]:
from transformers import TrainingArguments

training_args = TrainingArguments("test-trainer")

In [18]:
tokenized_datasets

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 8324
    })
    validation: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 1916
    })
    test: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 1518
    })
})

In [21]:
from transformers import Trainer

trainer = Trainer(
    model,
    training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics = compute_metrics
)

In [22]:
trainer.train()

Step,Training Loss
500,0.035800
1000,0.035200
1500,0.029700
2000,0.026100
2500,0.022400
3000,0.019900


TrainOutput(global_step=3123, training_loss=0.028216188090602595, metrics={'train_runtime': 494.001, 'train_samples_per_second': 50.551, 'train_steps_per_second': 6.322, 'total_flos': 899666963201448.0, 'train_loss': 0.028216188090602595, 'epoch': 3.0})

# Evaluate the fine-tuned model

## FILL IN code to evaluate performance of the model on the test set
Provide code to evaluate the pretrained model on the `test` portion of the dataset (`tokenized_datasets['test']`)

You'll need the F1 score for your report.
This is calculated automatically if you passed the `compute_metrics` function to the `Trainer` class.

In [24]:
# Testing code here
predictions = trainer.predict(tokenized_datasets["test"])

In [31]:
predictions[-1]

{'test_loss': 0.12180974334478378,
 'test_precision': 0.723716990533134,
 'test_recall': 0.8162405169991571,
 'test_f1': 0.7671992605308332,
 'test_accuracy': 0.9765721331689272,
 'test_runtime': 10.6394,
 'test_samples_per_second': 142.677,
 'test_steps_per_second': 17.858}

The F1 score for "chriskhanhtran/spanberta" is 0.7671992605308332

Hooray, you're done evaluating a model!

Feel free to restart the runtime and evaluate another one, or test that model on an example in the section below (which you'll need to do for at least one model).

# Test the model on an example
Code is provided here to test your fine-tuned classifier on an example sentence.

You will need to fill in the path to a checkpoint of your fine-tuned model if it has been saved somewhere. Or feel free to run your model some other way on the example sentence.

You will need the output of running at least one of your models on the example sentence for your report.

In [32]:
# Test performance on an example

from transformers import pipeline

checkpoint = "/content/test-trainer/checkpoint-3000" # FILL IN path to one of the checkpoints of your fine-tuned model
token_classifier = pipeline(
    "token-classification", model=checkpoint, aggregation_strategy="simple"
)

test_sentence = "Mi nombre is Miguel Salgado. Trabajo en la Universidad de Pittsburgh y vivo en Pittsburgh."
token_classifier(test_sentence)

[{'entity_group': 'PER',
  'score': 0.99776167,
  'word': ' Miguel Salgado.',
  'start': 13,
  'end': 28},
 {'entity_group': 'ORG',
  'score': 0.5067352,
  'word': ' Universidad',
  'start': 43,
  'end': 54},
 {'entity_group': 'LOC',
  'score': 0.82354796,
  'word': ' de Pittsburgh',
  'start': 55,
  'end': 68},
 {'entity_group': 'LOC',
  'score': 0.99871534,
  'word': ' Pittsburgh.',
  'start': 79,
  'end': 90}]

The model correctly identifies Miguel as a person, correctly identifies University as an organization, but not University of Pittsburgh, It labels de Pittsburgh and Pittsburgh as locations. So it is right if I say it has labelled an organization of a location, but not right if I want it to label the organization of a location as a singular organizational entity.

# Fine-tune models
This section is where you choose models and fill in parts of the code to do fine-tuning.

You need to fine-tune at least 2 pretrained models from the Hugging Face platform on the preprocessed CoNLL-2002 Spanish data:
* One BERT-based model pretrained with a regular masked language modeling (MLM) objective on a Spanish corpus. Examples: `PlanTL-GOB-ES/roberta-base-bne`, `chriskhanhtran/spanberta`
* One model pretrained to perform NER on another language, such as English. Models pretrained on the CoNLL-2003 dataset often work. Examples: `elastic/distilbert-base-cased-finetuned-conll03-english`, `dbmdz/bert-bert-cased-finetuned-conll03-english`

You'll want to make sure whatever pretrained model is cased, which contains valuable information for NER.

In [ ]:
# FILL IN which model you are fine-tuning and assign the name of it to the `pretrained_model` variable

pretrained_model = "chriskhanhtran/spanberta"
# pretrained_model = "elastic/distilbert-base-cased-finetuned-conll03-english"


In [ ]:
# Tokenize the data with the pretrained model's tokenizer
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained(pretrained_model, use_fast=True, add_prefix_space=True)

def align_labels_with_tokens(labels, word_ids):
  new_labels = []
  current_word=None
  for word_id in word_ids:
    if word_id != current_word:
      current_word = word_id
      label = -100 if word_id is None else labels[word_id]
      new_labels.append(label)

    elif word_id is None:
      new_labels.append(-100)

    else:
      label = labels[word_id]

      if label%2==1:
        label = label + 1
      new_labels.append(label)

  return new_labels

def tokenize_and_align_labels(examples):
  tokenized_inputs = tokenizer(examples['tokens'], truncation=True, is_split_into_words=True)

  all_labels = examples['ner_tags']

  new_labels = []
  for i, labels in enumerate(all_labels):
    word_ids = tokenized_inputs.word_ids(i)
    new_labels.append(align_labels_with_tokens(labels, word_ids))

  tokenized_inputs['labels'] = new_labels

  return tokenized_inputs
tokenized_datasets = data.map(tokenize_and_align_labels, batched=True, remove_columns=data['train'].column_names)
tokenized_datasets

Map:   0%|          | 0/8324 [00:00<?, ? examples/s]

Map:   0%|          | 0/1916 [00:00<?, ? examples/s]

Map:   0%|          | 0/1518 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 8324
    })
    validation: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 1916
    })
    test: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 1518
    })
})

In [ ]:
# Build a data collator to handle batching

from transformers import DataCollatorForTokenClassification

data_collator = DataCollatorForTokenClassification(tokenizer=tokenizer)

# Train (fine-tune) the model

In [ ]:
id2label = {i:label for i, label in enumerate(label_names)}
label2id = {label:i for i, label in enumerate(label_names)}
print(id2label)

{0: 'O', 1: 'B-PER', 2: 'I-PER', 3: 'B-ORG', 4: 'I-ORG', 5: 'B-LOC', 6: 'I-LOC', 7: 'B-MISC', 8: 'I-MISC'}


In [ ]:
from transformers import AutoModelForTokenClassification

model = AutoModelForTokenClassification.from_pretrained(pretrained_model,
                                                    id2label=id2label,
                                                    label2id=label2id)

Some weights of RobertaForTokenClassification were not initialized from the model checkpoint at chriskhanhtran/spanberta and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


## FILL IN code to train
Provide code to train (fine-tune) the pretrained model.
 You can use Hugging Face Trainer class or use any other package you want, such as PyTorch.

 See the [Hugging Face Trainer user guide](https://huggingface.co/learn/nlp-course/chapter3/3?fw=pt) or use any other online examples/resources you find online.

In [ ]:
from transformers import TrainingArguments

training_args = TrainingArguments("test-trainer")

In [ ]:
tokenized_datasets

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 8324
    })
    validation: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 1916
    })
    test: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 1518
    })
})

In [ ]:
from transformers import Trainer

trainer = Trainer(
    model,
    training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics = compute_metrics
)

In [ ]:
trainer.train()

Step,Training Loss
500,0.035800
1000,0.035200
1500,0.029700
2000,0.026100
2500,0.022400
3000,0.019900


TrainOutput(global_step=3123, training_loss=0.028216188090602595, metrics={'train_runtime': 494.001, 'train_samples_per_second': 50.551, 'train_steps_per_second': 6.322, 'total_flos': 899666963201448.0, 'train_loss': 0.028216188090602595, 'epoch': 3.0})

# Evaluate the fine-tuned model

## FILL IN code to evaluate performance of the model on the test set
Provide code to evaluate the pretrained model on the `test` portion of the dataset (`tokenized_datasets['test']`)

You'll need the F1 score for your report.
This is calculated automatically if you passed the `compute_metrics` function to the `Trainer` class.

In [ ]:
# Testing code here
predictions = trainer.predict(tokenized_datasets["test"])

In [ ]:
predictions[-1]

{'test_loss': 0.12180974334478378,
 'test_precision': 0.723716990533134,
 'test_recall': 0.8162405169991571,
 'test_f1': 0.7671992605308332,
 'test_accuracy': 0.9765721331689272,
 'test_runtime': 10.6394,
 'test_samples_per_second': 142.677,
 'test_steps_per_second': 17.858}

The F1 score for "chriskhanhtran/spanberta" is 0.7671992605308332

Hooray, you're done evaluating a model!

Feel free to restart the runtime and evaluate another one, or test that model on an example in the section below (which you'll need to do for at least one model).

# Test the model on an example
Code is provided here to test your fine-tuned classifier on an example sentence.

You will need to fill in the path to a checkpoint of your fine-tuned model if it has been saved somewhere. Or feel free to run your model some other way on the example sentence.

You will need the output of running at least one of your models on the example sentence for your report.

In [ ]:
# Test performance on an example

from transformers import pipeline

checkpoint = "/content/test-trainer/checkpoint-3000" # FILL IN path to one of the checkpoints of your fine-tuned model
token_classifier = pipeline(
    "token-classification", model=checkpoint, aggregation_strategy="simple"
)

test_sentence = "Mi nombre is Miguel Salgado. Trabajo en la Universidad de Pittsburgh y vivo en Pittsburgh."
token_classifier(test_sentence)

[{'entity_group': 'PER',
  'score': 0.99776167,
  'word': ' Miguel Salgado.',
  'start': 13,
  'end': 28},
 {'entity_group': 'ORG',
  'score': 0.5067352,
  'word': ' Universidad',
  'start': 43,
  'end': 54},
 {'entity_group': 'LOC',
  'score': 0.82354796,
  'word': ' de Pittsburgh',
  'start': 55,
  'end': 68},
 {'entity_group': 'LOC',
  'score': 0.99871534,
  'word': ' Pittsburgh.',
  'start': 79,
  'end': 90}]

The model correctly identifies Miguel as a person, correctly identifies University as an organization, but not University of Pittsburgh, It labels de Pittsburgh and Pittsburgh as locations. So it is right if I say it has labelled an organization of a location, but not right if I want it to label the organization of a location as a singular organizational entity.

Now I'll use the pretrained model in english

In [33]:
# FILL IN which model you are fine-tuning and assign the name of it to the `pretrained_model` variable

# pretrained_model = "chriskhanhtran/spanberta"
pretrained_model = "elastic/distilbert-base-cased-finetuned-conll03-english"

In [34]:
# Tokenize the data with the pretrained model's tokenizer
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained(pretrained_model, use_fast=True, add_prefix_space=True)

def align_labels_with_tokens(labels, word_ids):
  new_labels = []
  current_word=None
  for word_id in word_ids:
    if word_id != current_word:
      current_word = word_id
      label = -100 if word_id is None else labels[word_id]
      new_labels.append(label)

    elif word_id is None:
      new_labels.append(-100)

    else:
      label = labels[word_id]

      if label%2==1:
        label = label + 1
      new_labels.append(label)

  return new_labels

def tokenize_and_align_labels(examples):
  tokenized_inputs = tokenizer(examples['tokens'], truncation=True, is_split_into_words=True)

  all_labels = examples['ner_tags']

  new_labels = []
  for i, labels in enumerate(all_labels):
    word_ids = tokenized_inputs.word_ids(i)
    new_labels.append(align_labels_with_tokens(labels, word_ids))

  tokenized_inputs['labels'] = new_labels

  return tokenized_inputs
tokenized_datasets = data.map(tokenize_and_align_labels, batched=True, remove_columns=data['train'].column_names)
tokenized_datasets

Map:   0%|          | 0/8324 [00:00<?, ? examples/s]

Map:   0%|          | 0/1916 [00:00<?, ? examples/s]

Map:   0%|          | 0/1518 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 8324
    })
    validation: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 1916
    })
    test: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 1518
    })
})

In [35]:
# Build a data collator to handle batching

from transformers import DataCollatorForTokenClassification

data_collator = DataCollatorForTokenClassification(tokenizer=tokenizer)

# Train (fine-tune) the model

In [36]:
id2label = {i:label for i, label in enumerate(label_names)}
label2id = {label:i for i, label in enumerate(label_names)}
print(id2label)

{0: 'O', 1: 'B-PER', 2: 'I-PER', 3: 'B-ORG', 4: 'I-ORG', 5: 'B-LOC', 6: 'I-LOC', 7: 'B-MISC', 8: 'I-MISC'}


In [37]:
from transformers import AutoModelForTokenClassification

model = AutoModelForTokenClassification.from_pretrained(pretrained_model,
                                                    id2label=id2label,
                                                    label2id=label2id)

## FILL IN code to train
Provide code to train (fine-tune) the pretrained model.
 You can use Hugging Face Trainer class or use any other package you want, such as PyTorch.

 See the [Hugging Face Trainer user guide](https://huggingface.co/learn/nlp-course/chapter3/3?fw=pt) or use any other online examples/resources you find online.

In [38]:
from transformers import TrainingArguments

training_args = TrainingArguments("test-trainer")

In [39]:
tokenized_datasets

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 8324
    })
    validation: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 1916
    })
    test: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 1518
    })
})

In [40]:
from transformers import Trainer

trainer = Trainer(
    model,
    training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics = compute_metrics
)

In [41]:
trainer.train()

You're using a DistilBertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Step,Training Loss
500,0.165500
1000,0.103100
1500,0.066600
2000,0.057200
2500,0.037400
3000,0.029400


TrainOutput(global_step=3123, training_loss=0.07488215477314661, metrics={'train_runtime': 328.5481, 'train_samples_per_second': 76.007, 'train_steps_per_second': 9.505, 'total_flos': 770675224722480.0, 'train_loss': 0.07488215477314661, 'epoch': 3.0})

# Evaluate the fine-tuned model

## FILL IN code to evaluate performance of the model on the test set
Provide code to evaluate the pretrained model on the `test` portion of the dataset (`tokenized_datasets['test']`)

You'll need the F1 score for your report.
This is calculated automatically if you passed the `compute_metrics` function to the `Trainer` class.

In [42]:
# Testing code here
predictions = trainer.predict(tokenized_datasets["test"])

In [43]:
predictions[-1]

{'test_loss': 0.10008283704519272,
 'test_precision': 0.7569480859989512,
 'test_recall': 0.8111829165495926,
 'test_f1': 0.7831276278312763,
 'test_accuracy': 0.9762775712515489,
 'test_runtime': 7.7327,
 'test_samples_per_second': 196.309,
 'test_steps_per_second': 24.571}

The F1 score for "elastic/distilbert-base-cased-finetuned-conll03-english" is 0.7831276278312763

Hooray, you're done evaluating a model!

Feel free to restart the runtime and evaluate another one, or test that model on an example in the section below (which you'll need to do for at least one model).

# Test the model on an example
Code is provided here to test your fine-tuned classifier on an example sentence.

You will need to fill in the path to a checkpoint of your fine-tuned model if it has been saved somewhere. Or feel free to run your model some other way on the example sentence.

You will need the output of running at least one of your models on the example sentence for your report.

In [44]:
# Test performance on an example

from transformers import pipeline

checkpoint = "/content/test-trainer/checkpoint-3000" # FILL IN path to one of the checkpoints of your fine-tuned model
token_classifier = pipeline(
    "token-classification", model=checkpoint, aggregation_strategy="simple"
)

test_sentence = "Mi nombre is Miguel Salgado. Trabajo en la Universidad de Pittsburgh y vivo en Pittsburgh."
token_classifier(test_sentence)

[{'entity_group': 'PER',
  'score': 0.9935901,
  'word': 'Miguel Salgado. Trabajo',
  'start': 13,
  'end': 36},
 {'entity_group': 'ORG',
  'score': 0.88303494,
  'word': 'Universidad de Pittsburgh',
  'start': 43,
  'end': 68},
 {'entity_group': 'LOC',
  'score': 0.99703205,
  'word': 'Pittsburgh',
  'start': 79,
  'end': 89}]

Interestingly, this model corrects the University of Pittsburgh Organizational entity label I mentioned for the other model. It also captures the person's last name as well for the person label, which is really cool! However, it is a bit too liberal with the name recognition, because it also includes Trabajo in the person's name, which is a mistake. However, if the goal is to count the number of people referenced, then it is OK.